The goal for this piece of code is:
- Clean the data for the following datasets:
    - K2
    - APOKSAC
    - APOGEE
    - GALAH

After we have cleaned the datas is
- Use K2 model to predict the mass for APOGEE and GALAH
- Use APOKSAC model to predict mass for APOGEE

The goal is to get a table contain the following for all the data sets and use this to predict the mass of stars.
The table include parameters such as:

- Effective Temperature
- Log g
- Iron Abundance [Fe/H]
- Alpha Abundance [alpha/Fe]
- Carbon Abundance [C/Fe]
- Nitrogen Abundance [N/Fe]
- Oxygen Abundance [O/Fe]

In addition, we will extract the stars id to allow for identification

- `tmass_id` 2MASS IDs
- `gaiadr3_source_id` Gaia Id

Importing the required packages

In [4]:
from astropy.table import Table
import pandas as pd         # For table/data management

# Cleaning the GALAH Data

In [11]:
# Import the data and change into a data frame
def importGalah():
    galah = Table.read("RawData/galah_dr4_allspec_220713.fits", format = "fits")
    names = [name for name in galah.colnames if len(galah[name].shape) <= 1]
    galah = galah[names].to_pandas()
    return galah

In [12]:
galah = importGalah()

For Apogee Data, there are a few recommendations (this usually indicates that the data has some issues, e.g. unreliably broadening and other issues) for the flag that should be applied:
- `snr_c3_iraf > 30`
- `flag_sp == 0`
- For any element `X`, the flag `flag_X_fe == 0`

More information can be found [the GALAH Survey Website](https://www.galah-survey.org/dr3/flags/)

In [13]:
# Filter the data
# galah = galah.loc[galah["snr_c3_iraf"] > 30]
galah = galah.loc[galah["flag_sp"] == 0]
galah = galah.loc[galah["flag_fe_h"] == 0]
galah = galah.loc[galah["flag_al_fe"] == 0]
galah = galah.loc[galah["flag_c_fe"] == 0]
galah = galah.loc[galah["flag_n_fe"] == 0]
galah = galah.loc[galah["flag_o_fe"] == 0]

Now the appropriate flag has been used, we extract only the useful columns and save the data for later.

In [14]:
galah = galah[["teff", "logg", "fe_h", "al_fe", "c_fe", "n_fe", "o_fe"]]
# Remove all rows with null values
galah.dropna()
galah.to_csv("ProcessedData/cleaned_galah.csv", index=False)

# Cleaning APOGEE Data

In [8]:
def importApogee():
    apogee = Table.read("RawData/allStar-dr17-synspec_rev1.fits", format="fits", hdu=1)
    names = [name for name in apogee.colnames if len(apogee[name].shape) <= 1]
    apogee = apogee[names].to_pandas()
    return apogee

In [9]:
apogee = importApogee()

For APOGEE, we do something similar by making sure all the parameters are working on are good.
More information can be found [SDSS explanantion of bitmasks](https://www.sdss.org/dr13/algorithms/bitmasks/#APOGEE_ASPCAPFLAG) and [SDSS recommendations for bitmasks](https://www.sdss.org/dr13/irspec/abundances/)

The parameters
- `ASPCAPFLAG & STAR_BAD == 0` (`STAR_BAD` is the 23rd bit (starting from 0), or  `STAR_BAD = 0b 1000 0000 0000 0000 0000 0000`)
    - `STAR_BAD` is a general indication of whether there are issues with measuring surface temperature, surface gravity, and check if any elements have `GRIDEDGE_BAD` set
- `ELEMFLAG`
    - Any example of this is `FE_H_FLAG`
        - Make sure that the flag does not contain `GRIDEDGE_BAD`, `GRIDEDGE_WARN` and `CALRANGE_BAD`
    - We should just check that the flag is equal to zero to eliminate all and any problematic behaviour

In [12]:
# Filter the data
def cleanApogee(apogee):
    STAR_BAD = 1 << 23
    apogee = apogee.loc[apogee['ASPCAPFLAG'] & STAR_BAD == 0]
    apogee = apogee.loc[apogee['FE_H_FLAG'] == 0]
    apogee = apogee.loc[apogee['C_FE_FLAG'] == 0]
    apogee = apogee.loc[apogee['AL_FE_FLAG'] == 0]
    apogee = apogee.loc[apogee['N_FE_FLAG'] == 0]
    apogee = apogee.loc[apogee['O_FE_FLAG'] == 0]
    apogee.dropna()
    return apogee

In [13]:
apogee = cleanApogee(apogee)

Extract the useful column and save for later

In [14]:
apogee = apogee[["APOGEE_ID","TEFF", "LOGG", "FE_H", "AL_FE", "C_FE", "N_FE", "O_FE"]]
apogee.columns = apogee.columns.str.lower()
apogee.to_csv("ProcessedData/cleaned_apogee.csv", index=False)

# Cleaning APOKSAC 2 Data



In [47]:
apoksac_cols = ["kic", "tmass_id", "mass"]
apoksac_col_width = [(0, 9), (9, 26), (27, 39)]

apoksac = pd.read_fwf(
    "RawData/asu.tsv",
    skiprows=42,
    header=None,
    index_col=False,
    names = apoksac_cols,
    colspecs = apoksac_col_width
)
apoksac = apoksac[0:6676]
apoksac = apoksac.dropna()


,kic,tmass_id,mass
0,1027110,19250937+3644599,0.985
1,1027337,19252021+3647118,1.227
2,1160789,19233280+3652207,0.875
4,1161447,19241746+3651460,1.460
5,1161618,19242614+3648478,1.183
...,...,...,...
6671,12784948,19210624+5200236,1.254
6672,12785083,19212376+5204593,1.154
6673,12785250,19214766+5205365,1.386
6674,12884116,19182431+5215519,1.081


In [48]:
apoksac.to_csv("ProcessedData/cleaned_apoksac.csv", index=False)

Match APOKSAC data with APOGEE data

In [9]:
apogee = pd.read_csv("ProcessedData/cleaned_apogee.csv")
apogee = apogee[1:len(apogee)]

In [11]:
# Manipulate APOGEE id to 2MASS id
apogee = apogee.rename(columns = {"apogee_id": "tmass_id"})
apogee.tmass_id = apogee.tmass_id.str[4:-1]
apogee

,tmass_id,teff,logg,fe_h,al_fe,c_fe,n_fe,o_fe
1,00000035-7323394,4555.4043,1.498851,-1.171400,-0.455619,-0.290710,0.475530,0.152533
2,00000068+5710233,5031.2637,3.456132,-0.160280,0.014117,-0.025731,0.084176,0.113698
3,00000133+5721163,4974.6010,2.388251,-0.509530,0.158511,-0.069177,0.243080,0.185943
4,00000211+6327470,4681.4634,2.245527,-0.075011,-0.024401,-0.180636,0.345594,0.027783
5,00000222+5625359,5049.7686,3.291666,-0.139290,-0.074079,-0.011290,0.122410,0.087976
...,...,...,...,...,...,...,...,...
401156,23595676+7918072,4311.3926,1.731429,-0.504450,0.204981,0.094351,0.152260,0.287303
401157,23595756+8535275,5042.2480,4.507292,0.218410,0.086016,-0.036199,0.152240,-0.004475
401158,23595837+5700444,4954.1700,2.474665,-0.333100,0.139072,-0.096586,0.248020,0.130718
401159,23595886+5726058,4859.9087,2.480687,-0.236560,0.135759,-0.060314,0.201880,0.088913


# Cleaning K2 Data
To do so, we need to

1. extract the mass coefficient from the data set
2. find the effective temperature by finding the data in the GALAH data set by matching the EPIC identifier
3. determine the mass
4. extract the chemical composition of stars
5. store the data for use

For the purpose of this, we will keep the mass of stars as a multiple of solar mass.

In [ ]:
k2_cols = ["EPIC", "kappa-m-c"]
k2_col_width = [(0, 9), (24, 29)]

k2 = pd.read_fwf(
    "Data/k2_mass_coefficients.txt",
    skiprows=50,
    header=None,
    index_col=False,
    names = k2_cols,
    colspecs = k2_col_width
)
# k2.columns = k2_cols
k2
# According to the International Astronomical Union (IAU), effective temperature of the sun is 5772 +- 0.8K
teff_sun = 5772

To determine the mass for K2 stars, we use the following formula (from K2 Galactic Archaeology Program DR3):

$$ \frac{M}{M_\odot} = \kappa_M (\frac{T_{eff}}{T_{eff, \odot}})^{3/2} $$

To do so, we will use [gaia-kepler.fun](https://gaia-kepler.fun) in order to determine the link between 2MASS id and Kepler id.
- More information of the tables in the data set is given by [this documentation](https://exoplanetarchive.ipac.caltech.edu/docs/API_keplerstellar_columns.html)


In [ ]:
k2ToGaia = Table.read("RawData/kepler_dr3_good.fits", format = "fits")
names = [name for name in k2ToGaia.colnames if len(k2ToGaia[name].shape) <= 1]
k2ToGaia = k2ToGaia[names].to_pandas()

The

In [ ]:
k2ToGaia[["tm_designation", "kepid"]]